In [1]:
import sys
sys.path.append("..")
from preprocess.tile import tile_image
from smart_open import smart_open
import s3fs
from os import path, environ
import rasterio as rio
from re import match
import boto3

from tqdm import tqdm

from functools import partial

from concurrent import futures

environ['CURL_CA_BUNDLE']='/etc/ssl/certs/ca-certificates.crt'

# Tile Large Downloaded Images



In [2]:
# configuration
BUCKET = 'planet-snowcover-imagery'
AWS_PROFILE = 'esip'
IMG_REGEX = ".*_SR_.*\.tif"

In [3]:
# open s3 as filesystem with boto3 credentials
fs = s3fs.S3FileSystem(session= boto3.Session(profile_name = AWS_PROFILE))

In [4]:
# read files, pull out SR images 
files = fs.ls(BUCKET)
images = [f for f in files if match(IMG_REGEX, f)]

In [11]:
# tile all images (takes a while [~3m]). WRITES TO s3!
images = images[0:1]
outputs = []
for image in tqdm(images, total = len(images), unit='image'):
    imsize = fs.info(image, refresh=True)['Size'] 
    if (imsize == 0):
        print("skipping {}, (size:{})".format(image, imsize))
        continue #skip 
        
    try:
        
        outputdir = path.splitext(image)[0] + '_tiled'
        res = tile_image('s3://'+image,
                         's3://'+outputdir,
                         indexes=[1,2,3,4], zoom=15,
                         quant = 10000,
                         aws_profile = 'esip')
        outputs.append(res)
    except Exception as e:
        outputs.append(e)
        continue 
        


  0%|          | 0/1 [00:00<?, ?image/s]

invalid crs (epsg:32611), reprojecting raster....
reproject successful {'init': 'epsg:4326'}
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1000010000

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1000010000

1000010000

10000
10000
10000
10000
10000
10000
10000
1000010000

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1000010000

10000
1000010000
10000
10000

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1000010000
10000
10000

10000
10000
10000
10000
1000010000

10000
10000
10000
10000
10000
10000
1000010000

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1

100%|██████████| 1/1 [01:03<00:00, 63.03s/image]


In [49]:
outputs

[rasterio.errors.RasterioIOError("'/vsis3/planet-snowcover-imagery/20180601_181447_0f32_3B_AnalyticMS_SR_clip.tif' not recognized as a supported file format.")]

In [34]:
# check results
directories = [d for d in fs.ls(BUCKET, refresh=True) if match('.*_tiled', d)]

In [36]:
print('dir\ttiles')
for d in directories:
    print("{}\t{}".format(d, len(fs.walk(d))))

dir	tiles
planet-snowcover-imagery/20180601_181448_0f32_3B_AnalyticMS_SR_clip_tiled	366
planet-snowcover-imagery/20180601_181450_0f32_3B_AnalyticMS_SR_clip_tiled	366
planet-snowcover-imagery/20180601_181451_0f32_3B_AnalyticMS_SR_clip_tiled	378


In [51]:
fs.du('planet-snowcover-imagery/20180601_181447_0f32_3B_AnalyticMS_SR_clip.tif')

{'planet-snowcover-imagery/20180601_181447_0f32_3B_AnalyticMS_SR_clip.tif': 82562690}